In [1]:
import pandas as pd
import ast

In [2]:
def str_to_list(x):
    try:
        return list(ast.literal_eval(x))
    except:
        return None


def string_to_list_dataframe(df):
    columns = df.columns.tolist()
    columns_w_lists = []
    for column in columns:
        if df[column].astype(str). \
                apply(lambda x: x.startswith('[') and x.endswith(']')) \
                .astype(int).mean() > 0.8:
            columns_w_lists.append(column)
    for column in columns_w_lists:
        df[column] = df[column].apply(lambda x: str_to_list(x))
        df = df[~df[column].isna()]
    return df

# Load data

In [3]:
df = pd.read_csv('/home/web/files_for_research_Vova/processed_data.csv')

/home/web/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3221: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df = string_to_list_dataframe(df)

# See number of unique tokens

In [5]:
from itertools import chain

In [6]:
len(set(chain(*list(chain(*df['review_translate_sentences_lemma'].values.tolist())))))

205161

In [7]:
len(set(chain(*list(chain(*df['review_translate_sentences_tokens'].values.tolist())))))

437780

# Train BPE tokenizer

In [8]:
!ls /root

NVIDIA-Linux-x86_64-525.78.01.run  miniconda3  nltk_data  snap


In [9]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, Regex
import tokenizers
from nltk.tokenize import regexp_tokenize

In [10]:
df['review_translate'] = df['review_translate'].str.lower()

In [11]:
pre_tok = pre_tokenizers.Split(Regex(r"[\w'-]+|[^\w\s'-]+"),'removed', True)

In [28]:
bpe = models.BPE()


In [29]:
tokenizer = Tokenizer(model=bpe)
tokenizer.pre_tokenizer = pre_tok

In [45]:
trainer = trainers.BpeTrainer(special_tokens\
                              =["[UNK]", "[PAD]"],
                             min_frequency=5, end_of_word_suffix='</w>')

In [46]:
data = df['review_translate'].values.tolist()

In [47]:
tokenizer.train_from_iterator(data, trainer=trainer)

In [48]:
tokenizer.encode('привіт пан степан').tokens

['привіт</w>', 'пан</w>', 'сте', 'пан</w>']

In [49]:
tokenizer.get_vocab_size()

30000

# Save tokenizer

In [50]:
tokenizer.model.save(".")

['./vocab.json', './merges.txt']

In [51]:
!mv ./vocab.json /home/web/files_for_research_Vova/tokenizer_30k.json
!mv ./merges.txt /home/web/files_for_research_Vova/merges_tokenizer.txt


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
